# MAKING MOVIE RECOMMENDATE SYSTEM
- USING KAGGLE DATASET/
- ON CPU 
- LANGUAGE : python(3.12.7)
- libraries : requirements.txt
- similarity method not decided yet.
- After preprocessed working on unique movie data.

In [1]:
import warnings
warnings.filterwarnings("ignore")


In [3]:
import os 
import pandas as pd 
import numpy as np 

In [4]:
%pwd
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\HP\\Documents\\Movie_Recommendation_System'

In [18]:
data_path="Data/TMDB_all_movies.csv"
poster_path="https://image.tmdb.org/t/p/original/"

In [7]:
df=pd.read_csv(data_path)

# DATA UNDERSTANDING

In [8]:
df.head(1)

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path
0,2,Ariel,7.1,340.0,Released,1988-10-21,0.0,73.0,0.0,tt0094675,...,suomi,"Eetu Hilkamo, Turo Pajala, Jorma Markkula, Han...",Aki Kaurismäki,Timo Salminen,Aki Kaurismäki,Aki Kaurismäki,NaN,7.4,9066.0,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1073767 entries, 0 to 1073766
Data columns (total 28 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   id                       1073767 non-null  int64  
 1   title                    1073754 non-null  object 
 2   vote_average             1073765 non-null  float64
 3   vote_count               1073765 non-null  float64
 4   status                   1073765 non-null  object 
 5   release_date             951368 non-null   object 
 6   revenue                  1073765 non-null  float64
 7   runtime                  1073765 non-null  float64
 8   budget                   1073765 non-null  float64
 9   imdb_id                  610984 non-null   object 
 10  original_language        1073765 non-null  object 
 11  original_title           1073755 non-null  object 
 12  overview                 883928 non-null   object 
 13  popularity               1073765 non-null 

In [9]:
print("Shape of data is :",df.shape)

Shape of data is : (1073767, 28)


In [10]:
df["id"].nunique()

1073767

In [11]:
df["imdb_id"].nunique()

609836

In [12]:
print("Total Null Values in DataSet :", df.isnull().sum().sum())

Total Null Values in DataSet : 8566316


# DATA PREPROCESSING 

In [13]:
# droping NULL VALUES 
df.dropna(inplace=True)

In [14]:
print("*"*5+"After Removing Null Values"+"*"*5)
print("Shape of data  is :",df.shape)

*****After Removing Null Values*****
Shape of data  is : (26820, 28)


In [15]:
print("Duplicate values in data is :",df.duplicated().sum())

Duplicate values in data is : 0


In [16]:
print("Unique id:",df["id"].nunique())
print("Unique Imdb id:",df["imdb_id"].nunique())

Unique id: 26820
Unique Imdb id: 26820


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26820 entries, 2 to 1060933
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       26820 non-null  int64  
 1   title                    26820 non-null  object 
 2   vote_average             26820 non-null  float64
 3   vote_count               26820 non-null  float64
 4   status                   26820 non-null  object 
 5   release_date             26820 non-null  object 
 6   revenue                  26820 non-null  float64
 7   runtime                  26820 non-null  float64
 8   budget                   26820 non-null  float64
 9   imdb_id                  26820 non-null  object 
 10  original_language        26820 non-null  object 
 11  original_title           26820 non-null  object 
 12  overview                 26820 non-null  object 
 13  popularity               26820 non-null  float64
 14  tagline                  

In [30]:
df.describe()

,id,vote_average,vote_count,revenue,runtime,budget,popularity,imdb_rating,imdb_votes
count,2.682000e+04,26820.000000,26820.000000,2.682000e+04,26820.000000,2.682000e+04,26820.000000,26820.000000,2.682000e+04
mean,2.209563e+05,5.871432,662.912975,2.309283e+07,100.659135,8.621847e+06,6.436539,6.037591,3.522963e+04
std,2.915216e+05,1.380183,2053.721856,9.633021e+07,22.930249,2.583515e+07,9.119699,1.141163,1.169144e+05
min,5.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,1.300000,5.000000e+00
25%,2.556150e+04,5.300000,15.000000,0.000000e+00,89.000000,0.000000e+00,2.331750,5.400000,7.630000e+02
50%,7.724200e+04,6.100000,62.000000,0.000000e+00,97.000000,0.000000e+00,3.967000,6.200000,3.259500e+03
75%,3.290055e+05,6.700000,338.250000,3.603762e+06,110.000000,3.500000e+06,7.985500,6.800000,1.765400e+04
max,1.440153e+06,10.000000,37198.000000,2.923706e+09,803.000000,4.600000e+08,459.580500,9.800000,3.023533e+06


In [19]:
df.head(1)

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path
2,5,Four Rooms,5.9,2681.0,Released,1995-12-09,4257354.0,98.0,4000000.0,tt0113101,...,English,"Antonio Banderas, Sammi Davis, Kimberly Blair,...","Allison Anders, Quentin Tarantino, Robert Rodr...","Andrzej Sekula, Phil Parmet, Guillermo Navarro...","Allison Anders, Quentin Tarantino, Robert Rodr...","Alexandre Rockwell, Lawrence Bender, Quentin T...",Combustible Edison,6.7,113781.0,/pyCk5JgtRZwRxnXwfrvyzukaKue.jpg


# REMOVING UNWANTED COLUMNS

In [20]:
df.columns

Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'budget', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'cast', 'director', 'director_of_photography', 'writers', 'producers',
       'music_composer', 'imdb_rating', 'imdb_votes', 'poster_path'],
      dtype='object')

In [21]:
df["original_language"].value_counts()

original_language
en    21007
it      716
ja      621
fr      595
ko      553
      ...  
su        1
dz        1
or        1
ga        1
be        1
Name: count, Length: 80, dtype: int64

In [22]:
(len(df[df["original_language"]=="en"]))/len(df)*100

78.32587621178226

In [23]:
df["genres"]

2                                   Comedy, Crime
3                         Action, Crime, Thriller
6              Adventure, Action, Science Fiction
7                               Animation, Family
8                          Comedy, Drama, Romance
                            ...                  
1041219                                    Horror
1049001                           Romance, Comedy
1051552    Science Fiction, Action, Thriller, War
1060661                                     Drama
1060933                                     Drama
Name: genres, Length: 26820, dtype: object

In [24]:
# in original language en is more than 75 %  
# as in recommendation task we dont want unusable numeric column so we can remove it
# in original_title title can be non english so that we remove it and use title column for english title in whole dataset.

In [ ]:
# important columns 
# - id (will use to retrive specific metadata)
# - title (will use to get the title of movie)
# - overview ( use for content based recommendation system )
# - cast ( use for content based recommendation system )
# - director ( use for content based recommendation system )
# - generes  ( use for content based recommendation system )

In [26]:
updated_df=df[["id","title","overview","cast","director","genres"]]

In [27]:
updated_df

,id,title,overview,cast,director,genres
2,5,Four Rooms,It's Ted the Bellhop's first night on the job....,"Antonio Banderas, Sammi Davis, Kimberly Blair,...","Allison Anders, Quentin Tarantino, Robert Rodr...","Comedy, Crime"
3,6,Judgment Night,"Four young friends, while taking a shortcut en...","Stephen Dorff, Everlast, Will Zahrn, Emilio Es...",Stephen Hopkins,"Action, Crime, Thriller"
6,11,Star Wars,Princess Leia is captured and held hostage by ...,"David Ankrum, Mark Hamill, Tim Condren, Frank ...",George Lucas,"Adventure, Action, Science Fiction"
7,12,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...","Jeff Pidgeon, Kali Whitehurst, Danny Mann, Lar...",Andrew Stanton,"Animation, Family"
8,13,Forrest Gump,A man with a low IQ has accomplished great thi...,"Zach Hanner, Conor Kennelly, Elvis Presley, Ru...",Robert Zemeckis,"Comedy, Drama, Romance"
...,...,...,...,...,...,...
1041219,1416131,Steps,"When Molly moves into a new house, she soon di...","Mori Christian, Kris J. Cummins",Russell Miller,Horror
1049001,1426142,Sosyal Climbers,Stuck with a massive debt after falling for a ...,"Jan Silverio, Maris Racal, Gemma Masas, Bart G...",Jason Paul Laxamana,"Romance, Comedy"
1051552,1429172,Halo4ForwardUntoDawn,Set against the backdrop of a United Nations S...,"Tony Giroux, Daniel Cudmore, Sean Carey, Darre...",Stewart Hendler,"Science Fiction, Action, Thriller, War"
1060661,1439845,L'amore che vorrei,Five women are victims of various forms of vio...,"Michelle Hunziker, Claudia Potenza, Michela An...",Gabriele Pignotta,Drama


# FIX SOME FORMATS AND MAKE DATA SUITABLE FOR VECTORIZATION 

In [28]:
# 1. take top 3 cast from cast column
# 2. take top 1 director from director column 
# 3. remove space between cast name and director name  
#   - so that when we make tags for recommendation we wont have confusion 
# 4. take all generes and put it into list 
# 5. merge overview , cast, director and make 1 single column named tags.

In [29]:
updated_df["cast"]=updated_df["cast"].apply(lambda x :   x.split(",")[:3])

In [30]:
updated_df["director"]=updated_df["director"].apply(lambda x :   x.split(",")[:1])

In [31]:
updated_df.head()

,id,title,overview,cast,director,genres
2,5,Four Rooms,It's Ted the Bellhop's first night on the job....,"[Antonio Banderas, Sammi Davis, Kimberly Blair]",[Allison Anders],"Comedy, Crime"
3,6,Judgment Night,"Four young friends, while taking a shortcut en...","[Stephen Dorff, Everlast, Will Zahrn]",[Stephen Hopkins],"Action, Crime, Thriller"
6,11,Star Wars,Princess Leia is captured and held hostage by ...,"[David Ankrum, Mark Hamill, Tim Condren]",[George Lucas],"Adventure, Action, Science Fiction"
7,12,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...","[Jeff Pidgeon, Kali Whitehurst, Danny Mann]",[Andrew Stanton],"Animation, Family"
8,13,Forrest Gump,A man with a low IQ has accomplished great thi...,"[Zach Hanner, Conor Kennelly, Elvis Presley]",[Robert Zemeckis],"Comedy, Drama, Romance"


In [32]:
def remove_space(x):
    return x.replace(" ","")

In [33]:
updated_cast=[]
for i,val in enumerate(updated_df["cast"]) :
    updated_cast.append([])
    for j in val:
        updated_cast[i].append(remove_space(j))

In [34]:
updated_df["cast"]=updated_cast

In [35]:
updated_df

,id,title,overview,cast,director,genres
2,5,Four Rooms,It's Ted the Bellhop's first night on the job....,"[AntonioBanderas, SammiDavis, KimberlyBlair]",[Allison Anders],"Comedy, Crime"
3,6,Judgment Night,"Four young friends, while taking a shortcut en...","[StephenDorff, Everlast, WillZahrn]",[Stephen Hopkins],"Action, Crime, Thriller"
6,11,Star Wars,Princess Leia is captured and held hostage by ...,"[DavidAnkrum, MarkHamill, TimCondren]",[George Lucas],"Adventure, Action, Science Fiction"
7,12,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...","[JeffPidgeon, KaliWhitehurst, DannyMann]",[Andrew Stanton],"Animation, Family"
8,13,Forrest Gump,A man with a low IQ has accomplished great thi...,"[ZachHanner, ConorKennelly, ElvisPresley]",[Robert Zemeckis],"Comedy, Drama, Romance"
...,...,...,...,...,...,...
1041219,1416131,Steps,"When Molly moves into a new house, she soon di...","[MoriChristian, KrisJ.Cummins]",[Russell Miller],Horror
1049001,1426142,Sosyal Climbers,Stuck with a massive debt after falling for a ...,"[JanSilverio, MarisRacal, GemmaMasas]",[Jason Paul Laxamana],"Romance, Comedy"
1051552,1429172,Halo4ForwardUntoDawn,Set against the backdrop of a United Nations S...,"[TonyGiroux, DanielCudmore, SeanCarey]",[Stewart Hendler],"Science Fiction, Action, Thriller, War"
1060661,1439845,L'amore che vorrei,Five women are victims of various forms of vio...,"[MichelleHunziker, ClaudiaPotenza, MichelaAndr...",[Gabriele Pignotta],Drama


In [36]:
updated_director=[]
for i,val in enumerate(updated_df["director"]) :
    updated_director.append([])
    for j in val:
        updated_director[i].append(remove_space(j))

In [37]:
updated_df["director"]=updated_director

In [38]:
updated_df

,id,title,overview,cast,director,genres
2,5,Four Rooms,It's Ted the Bellhop's first night on the job....,"[AntonioBanderas, SammiDavis, KimberlyBlair]",[AllisonAnders],"Comedy, Crime"
3,6,Judgment Night,"Four young friends, while taking a shortcut en...","[StephenDorff, Everlast, WillZahrn]",[StephenHopkins],"Action, Crime, Thriller"
6,11,Star Wars,Princess Leia is captured and held hostage by ...,"[DavidAnkrum, MarkHamill, TimCondren]",[GeorgeLucas],"Adventure, Action, Science Fiction"
7,12,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...","[JeffPidgeon, KaliWhitehurst, DannyMann]",[AndrewStanton],"Animation, Family"
8,13,Forrest Gump,A man with a low IQ has accomplished great thi...,"[ZachHanner, ConorKennelly, ElvisPresley]",[RobertZemeckis],"Comedy, Drama, Romance"
...,...,...,...,...,...,...
1041219,1416131,Steps,"When Molly moves into a new house, she soon di...","[MoriChristian, KrisJ.Cummins]",[RussellMiller],Horror
1049001,1426142,Sosyal Climbers,Stuck with a massive debt after falling for a ...,"[JanSilverio, MarisRacal, GemmaMasas]",[JasonPaulLaxamana],"Romance, Comedy"
1051552,1429172,Halo4ForwardUntoDawn,Set against the backdrop of a United Nations S...,"[TonyGiroux, DanielCudmore, SeanCarey]",[StewartHendler],"Science Fiction, Action, Thriller, War"
1060661,1439845,L'amore che vorrei,Five women are victims of various forms of vio...,"[MichelleHunziker, ClaudiaPotenza, MichelaAndr...",[GabrielePignotta],Drama


In [39]:
updated_df["genres"]=updated_df["genres"].apply(lambda x : x.split(","))

In [40]:
updated_df["overview"]=updated_df["overview"].apply(lambda x:x.split())

In [41]:
updated_df

,id,title,overview,cast,director,genres
2,5,Four Rooms,"[It's, Ted, the, Bellhop's, first, night, on, ...","[AntonioBanderas, SammiDavis, KimberlyBlair]",[AllisonAnders],"[Comedy, Crime]"
3,6,Judgment Night,"[Four, young, friends,, while, taking, a, shor...","[StephenDorff, Everlast, WillZahrn]",[StephenHopkins],"[Action, Crime, Thriller]"
6,11,Star Wars,"[Princess, Leia, is, captured, and, held, host...","[DavidAnkrum, MarkHamill, TimCondren]",[GeorgeLucas],"[Adventure, Action, Science Fiction]"
7,12,Finding Nemo,"[Nemo,, an, adventurous, young, clownfish,, is...","[JeffPidgeon, KaliWhitehurst, DannyMann]",[AndrewStanton],"[Animation, Family]"
8,13,Forrest Gump,"[A, man, with, a, low, IQ, has, accomplished, ...","[ZachHanner, ConorKennelly, ElvisPresley]",[RobertZemeckis],"[Comedy, Drama, Romance]"
...,...,...,...,...,...,...
1041219,1416131,Steps,"[When, Molly, moves, into, a, new, house,, she...","[MoriChristian, KrisJ.Cummins]",[RussellMiller],[Horror]
1049001,1426142,Sosyal Climbers,"[Stuck, with, a, massive, debt, after, falling...","[JanSilverio, MarisRacal, GemmaMasas]",[JasonPaulLaxamana],"[Romance, Comedy]"
1051552,1429172,Halo4ForwardUntoDawn,"[Set, against, the, backdrop, of, a, United, N...","[TonyGiroux, DanielCudmore, SeanCarey]",[StewartHendler],"[Science Fiction, Action, Thriller, War]"
1060661,1439845,L'amore che vorrei,"[Five, women, are, victims, of, various, forms...","[MichelleHunziker, ClaudiaPotenza, MichelaAndr...",[GabrielePignotta],[Drama]


In [42]:
updated_df["tags"]=updated_df["overview"]+updated_df["cast"]+updated_df["director"]+updated_df["genres"]

In [43]:
updated_df["tags"].iloc[0]

["It's",
 'Ted',
 'the',
 "Bellhop's",
 'first',
 'night',
 'on',
 'the',
 'job...and',
 'the',
 "hotel's",
 'very',
 'unusual',
 'guests',
 'are',
 'about',
 'to',
 'place',
 'him',
 'in',
 'some',
 'outrageous',
 'predicaments.',
 'It',
 'seems',
 'that',
 'this',
 "evening's",
 'room',
 'service',
 'is',
 'serving',
 'up',
 'one',
 'unbelievable',
 'happening',
 'after',
 'another.',
 'AntonioBanderas',
 'SammiDavis',
 'KimberlyBlair',
 'AllisonAnders',
 'Comedy',
 ' Crime']

In [44]:
updated_df["tags"]= updated_df["tags"].apply(lambda x : " ".join(x))

In [45]:
updated_df

,id,title,overview,cast,director,genres,tags
2,5,Four Rooms,"[It's, Ted, the, Bellhop's, first, night, on, ...","[AntonioBanderas, SammiDavis, KimberlyBlair]",[AllisonAnders],"[Comedy, Crime]",It's Ted the Bellhop's first night on the job....
3,6,Judgment Night,"[Four, young, friends,, while, taking, a, shor...","[StephenDorff, Everlast, WillZahrn]",[StephenHopkins],"[Action, Crime, Thriller]","Four young friends, while taking a shortcut en..."
6,11,Star Wars,"[Princess, Leia, is, captured, and, held, host...","[DavidAnkrum, MarkHamill, TimCondren]",[GeorgeLucas],"[Adventure, Action, Science Fiction]",Princess Leia is captured and held hostage by ...
7,12,Finding Nemo,"[Nemo,, an, adventurous, young, clownfish,, is...","[JeffPidgeon, KaliWhitehurst, DannyMann]",[AndrewStanton],"[Animation, Family]","Nemo, an adventurous young clownfish, is unexp..."
8,13,Forrest Gump,"[A, man, with, a, low, IQ, has, accomplished, ...","[ZachHanner, ConorKennelly, ElvisPresley]",[RobertZemeckis],"[Comedy, Drama, Romance]",A man with a low IQ has accomplished great thi...
...,...,...,...,...,...,...,...
1041219,1416131,Steps,"[When, Molly, moves, into, a, new, house,, she...","[MoriChristian, KrisJ.Cummins]",[RussellMiller],[Horror],"When Molly moves into a new house, she soon di..."
1049001,1426142,Sosyal Climbers,"[Stuck, with, a, massive, debt, after, falling...","[JanSilverio, MarisRacal, GemmaMasas]",[JasonPaulLaxamana],"[Romance, Comedy]",Stuck with a massive debt after falling for a ...
1051552,1429172,Halo4ForwardUntoDawn,"[Set, against, the, backdrop, of, a, United, N...","[TonyGiroux, DanielCudmore, SeanCarey]",[StewartHendler],"[Science Fiction, Action, Thriller, War]",Set against the backdrop of a United Nations S...
1060661,1439845,L'amore che vorrei,"[Five, women, are, victims, of, various, forms...","[MichelleHunziker, ClaudiaPotenza, MichelaAndr...",[GabrielePignotta],[Drama],Five women are victims of various forms of vio...


In [46]:
structured_df=updated_df[["id","title","tags"]]

In [47]:
structured_df

,id,title,tags
2,5,Four Rooms,It's Ted the Bellhop's first night on the job....
3,6,Judgment Night,"Four young friends, while taking a shortcut en..."
6,11,Star Wars,Princess Leia is captured and held hostage by ...
7,12,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp..."
8,13,Forrest Gump,A man with a low IQ has accomplished great thi...
...,...,...,...
1041219,1416131,Steps,"When Molly moves into a new house, she soon di..."
1049001,1426142,Sosyal Climbers,Stuck with a massive debt after falling for a ...
1051552,1429172,Halo4ForwardUntoDawn,Set against the backdrop of a United Nations S...
1060661,1439845,L'amore che vorrei,Five women are victims of various forms of vio...


In [49]:
structured_df["tags"]=structured_df["tags"].apply(lambda x:x.lower())

In [50]:
structured_df

,id,title,tags
2,5,Four Rooms,it's ted the bellhop's first night on the job....
3,6,Judgment Night,"four young friends, while taking a shortcut en..."
6,11,Star Wars,princess leia is captured and held hostage by ...
7,12,Finding Nemo,"nemo, an adventurous young clownfish, is unexp..."
8,13,Forrest Gump,a man with a low iq has accomplished great thi...
...,...,...,...
1041219,1416131,Steps,"when molly moves into a new house, she soon di..."
1049001,1426142,Sosyal Climbers,stuck with a massive debt after falling for a ...
1051552,1429172,Halo4ForwardUntoDawn,set against the backdrop of a united nations s...
1060661,1439845,L'amore che vorrei,five women are victims of various forms of vio...


# VECTORIZATION (BAG OF WORDS) CAN ALSO USE WORD2VEC, tifz

In [ ]:
structured_df.reset_index(inplace=True)
structured_df.drop(["index"],axis=1,inplace=True)

In [ ]:
from nltk.stem import PorterStemmer
# we use Stemmer for making different tense word to root word 
# e.g [loved,love,loving]-> love

In [53]:
ps=PorterStemmer()

In [54]:
def stemmer(text):
    y=[]
    for  i in text.split(" "):
        y.append(ps.stem(i))
    return " ".join(y)


In [58]:
structured_df["tags"]=structured_df["tags"].apply(stemmer)

In [59]:
from sklearn.feature_extraction.text import CountVectorizer

In [68]:
cv=CountVectorizer(max_features=5000,stop_words="english")

In [69]:
vectors=cv.fit_transform(structured_df["tags"]).toarray()

In [71]:
len(cv.get_feature_names_out())

5000

In [76]:
from sklearn.metrics.pairwise import cosine_similarity

array([[1.        , 0.06063391, 0.        , ..., 0.03008284, 0.        ,
        0.        ],
       [0.06063391, 1.        , 0.04351941, ..., 0.06201737, 0.        ,
        0.        ],
       [0.        , 0.04351941, 1.        , ..., 0.06477503, 0.        ,
        0.        ],
       ...,
       [0.03008284, 0.06201737, 0.06477503, ..., 1.        , 0.03008284,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.03008284, 1.        ,
        0.08574929],
       [0.        , 0.        , 0.        , ..., 0.        , 0.08574929,
        1.        ]], shape=(26820, 26820))

In [ ]:
distances=cosine_similarity(vectors)

In [110]:
distances.shape

(26820, 26820)

# MAIN FUNCTION FOR RETRIVING RECOMMENDATE MOVIES

In [193]:
def recommendate_system(movie_name):
    movie_index=(df[df["title"]==movie_name]).index[0]
    distance_vector=top_50_distance[movie_index]
    recommendate_movies=distance_vector[0:5]
    
    movie_names=[]

    for i in recommendate_movies:
        print(df.iloc[i[0]]["title"])
        
    

In [194]:
recommendate_system("Star Wars")

The Empire Strikes Back
Star Wars: The Force Awakens
Samson and the 7 Miracles of the World
Return of the Jedi
The Star Wars Holiday Special


In [143]:
recommendate_system("Star Wars")

The Empire Strikes Back
Star Wars: The Force Awakens
Samson and the 7 Miracles of the World
Return of the Jedi
The Star Wars Holiday Special


In [ ]:
#store top 50 movies similiar for each movie for reducing size from 26820 x 26820 to 26820 x 50

top_50_distance=[]
for i in distances:
    top_50_distance.append(sorted(list(enumerate(i)),reverse=True,key=lambda x:x[1])[1:51])

# STORE IMPORTANT HELPERS FOR MOVIE RECOMMENDATION IN PKL FORMAT

In [170]:
from joblib import dump
dump(top_50_distance, "model_helpers/similarity.pkl")


['model_helpers/similarity.pkl']

In [173]:
df.reset_index(inplace=True)
df.drop(["index"],axis=1,inplace=True)

In [175]:
dump(df, "model_helpers/movie_metadata.pkl")

['model_helpers/movie_metadata.pkl']

In [195]:
df.to_csv("Data/TMDB_MOVIE_SUBDATASET.csv")

# END 
<hr/>